In [ ]:
import cv2
import mediapipe as mp
import csv
import pandas as pd
import os

In [ ]:
# Initialize MediaPipe Pose and Drawing utilities
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

In [ ]:
DATASET_DIR = 'Single_person_violent'
CLASSES_LIST = ["Kicking","Punching"]
output_path = 'Output'

In [153]:
def write_landmarks_to_csv(landmarks, frame_number, csv_data):
    #print(f"Landmark coordinates for frame {frame_number}:")
    for idx, landmark in enumerate(landmarks):
        #print(f"{mp_pose.PoseLandmark(idx).name}: (x: {landmark.x}, y: {landmark.y}, z: {landmark.z})")
        csv_data.append([frame_number, landmark.x, landmark.y, landmark.z, landmark.visibility])    

In [154]:
def convert_video_to_landmark_csv(video_path):
    frame_number = 0
    csv_data = []

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            frame_number += 1
            ret, frame = cap.read()
            if not ret:
                break

            # Convert the image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False

            # Perform pose detection
            results = pose.process(image)

            # Convert the image back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw the pose annotation on the image
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(
                    image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            # Add the landmark coordinates to the list and print them
            write_landmarks_to_csv(results.pose_landmarks.landmark, frame_number, csv_data)   
    cap.release() 
    # Write the 2D array to a CSV file
    file_name = video_path.split('\\')[-1][:-4]
    class_name = video_path.split('\\')[-2]    
    output_file = os.path.join(output_path, class_name,file_name)    
    with open(f'{output_file}.csv', mode='w', newline='') as file:
        writer = csv.writer(file)
        # Write each row of the 2D array
        for row in csv_data:
            writer.writerow(row)

In [ ]:
def create_dataset(CLASSES_LIST):
    print('List: ',CLASSES_LIST)
    for class_name in CLASSES_LIST:
        print(f"Extracting data from {class_name}")
        # Get list of videos for each class
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        for file_name in files_list:
            # Get the complete video path.
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
            convert_video_to_landmark_csv(video_file_path)

In [ ]:
create_dataset(CLASSES_LIST)

In [ ]:
custom_headers = ['Frame Number', 'x','y','z','Visibility']
df = pd.read_csv('output.csv', header=None, names=custom_headers)

In [ ]:
df.head()

In [ ]:
def createframe(csv_path):
    custom_headers = ['Frame Number', 'x', 'y', 'z', 'Visibility']
    data = pd.read_csv(csv_path, header=None, names=custom_headers)
    #frame = [[d[1]['x'], d[1]['y'], d[1]['z'], d[1]['Visibility']] for d in data.iterrows()]
    current_frame = -1
    frame = []
    sequence = []
    for d in data.iterrows():
        if not current_frame == d[1]['Frame Number']:
            current_frame = d[1]['Frame Number']
            if frame:              
                sequence.append(frame)                            
                frame = []        
        frame.append([d[1]['x'], d[1]['y'], d[1]['z'], d[1]['Visibility']])    
    sequence.append(frame)            
    return sequence


In [ ]:
# Read data from CSV and append to processed_df
frame = createframe('output.csv')
processed_df = pd.DataFrame([frame])
#processed_df2 = processed_df.append(pd.DataFrame(frame, columns=['x', 'y', 'z', 'Visibility']), ignore_index=True)